In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sqlalchemy import create_engine

In [2]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/unicef_db", echo=True)

In [15]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. Lire les fichiers Excel
# Remplacer 'fichier1.xlsx' et 'fichier2.xlsx' par vos noms de fichiers réels.
df1 = pd.read_excel('Datasets/Maladies_Couv_Sante.xlsx')  # Colonnes: index, district, code_region, maladie
df2 = pd.read_excel('Datasets/REGION.xlsx')  # Colonnes: index, code, nom_region

# Optionnel : renommer les colonnes si besoin pour éviter les conflits avec des noms réservés
df1 = df1.rename(columns={"index": "id_Maladie"})
df2 = df2.rename(columns={"index": "id_Region"})



# 4. Charger les données dans les tables
# On insère d'abord les données de la table 'region' (fichier 2)
df2.to_sql('region', engine, if_exists='replace', index=False)

# Puis on insère les données de la table 'maladie' (fichier 1)
# On garde les colonnes pertinentes et on s'assure que la colonne 'code_region' correspond bien
df1.to_sql('maladie', engine, if_exists='append', index=False)


2025-03-11 13:37:19,275 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-11 13:37:19,279 INFO sqlalchemy.engine.Engine DESCRIBE `unicef_db`.`region`
2025-03-11 13:37:19,281 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:19,309 INFO sqlalchemy.engine.Engine DESCRIBE `unicef_db`.`region`
2025-03-11 13:37:19,311 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:19,326 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `unicef_db`
2025-03-11 13:37:19,328 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:19,332 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `unicef_db`
2025-03-11 13:37:19,334 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:19,339 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `region`
2025-03-11 13:37:19,340 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:19,346 INFO sqlalchemy.engine.Engine 
DROP TABLE region
2025-03-11 13:37:19,348 INFO sqlalchemy.engine.Engine [no key 0.00150s] {}
2025-03-11 13:37:19,5

792

In [16]:
query = """
SELECT *
FROM maladie m
JOIN region r ON m.CODE_REGION = r.CODE_REGION
"""
df_maladie = pd.read_sql(query, engine)

2025-03-11 13:37:20,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-11 13:37:20,478 INFO sqlalchemy.engine.Engine DESCRIBE `unicef_db`.`
SELECT *
FROM maladie m
JOIN region r ON m.CODE_REGION = r.CODE_REGION
`
2025-03-11 13:37:20,480 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:20,491 INFO sqlalchemy.engine.Engine 
SELECT *
FROM maladie m
JOIN region r ON m.CODE_REGION = r.CODE_REGION

2025-03-11 13:37:20,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-11 13:37:20,979 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
df_maladie

,index,CODE_DISTRICT,district,CODE_REGION,Mois,Année,IRA,Diarrhée,Palu,CODE_REGION,LIBELLE_REGION
0,0.0,533,Amboasary Atsimo,53,1,2021,333.0,1363.0,3262.0,53,ANOSY
1,1.0,533,Amboasary Atsimo,53,2,2021,153.0,1046.0,2487.0,53,ANOSY
2,2.0,533,Amboasary Atsimo,53,3,2021,343.0,1082.0,4324.0,53,ANOSY
3,3.0,533,Amboasary Atsimo,53,4,2021,156.0,788.0,4218.0,53,ANOSY
4,4.0,533,Amboasary Atsimo,53,5,2021,160.0,547.0,6255.0,53,ANOSY
...,...,...,...,...,...,...,...,...,...,...,...
3955,NaN,515,Sakaraha,51,8,2023,116.0,211.0,2573.0,51,ATSIMO ANDREFANA
3956,NaN,515,Sakaraha,51,9,2023,214.0,229.0,2774.0,51,ATSIMO ANDREFANA
3957,NaN,515,Sakaraha,51,10,2023,116.0,314.0,3182.0,51,ATSIMO ANDREFANA
3958,NaN,515,Sakaraha,51,11,2023,NaN,NaN,NaN,51,ATSIMO ANDREFANA


In [32]:
anne_mal =   df_maladie.groupby("Année")[["IRA", "Palu", "Diarrhée"]].sum()
anne_mal

,IRA,Palu,Diarrhée
Année,,,
2021,693720.0,4468330.0,580485.0
2022,798075.0,3595300.0,619805.0
2023,751040.0,9177900.0,561895.0


In [33]:
anne_mal.values.tolist()

[[693720.0, 4468330.0, 580485.0],
 [798075.0, 3595300.0, 619805.0],
 [751040.0, 9177900.0, 561895.0]]

In [34]:
anne_mal.index.tolist()

[2021, 2022, 2023]

In [55]:
df_filtered = df_maladie[df_maladie["Année"] == 2022]

In [56]:
result = df_filtered.groupby("LIBELLE_REGION")[["IRA", "Diarrhée", "Palu"]].sum().reset_index()

In [59]:
total = []
maladie = ["IRA", "Palu", "Diarrhée"]

for val in maladie:
    som = df_filtered[val].sum()
    total.append(som)    

generale_annee = {key: value for key, value in zip(maladie, total)}

print(generale_annee)

{'IRA': 798075.0, 'Palu': 3595300.0, 'Diarrhée': 619805.0}


In [57]:
result.to_dict(orient="records")

[{'LIBELLE_REGION': 'ANDROY',
  'IRA': 65660.0,
  'Diarrhée': 81815.0,
  'Palu': 47670.0},
 {'LIBELLE_REGION': 'ANOSY',
  'IRA': 139340.0,
  'Diarrhée': 107840.0,
  'Palu': 830395.0},
 {'LIBELLE_REGION': 'ATSIMO ANDREFANA',
  'IRA': 106745.0,
  'Diarrhée': 73890.0,
  'Palu': 287255.0},
 {'LIBELLE_REGION': 'ATSIMO ATSINANANA',
  'IRA': 255550.0,
  'Diarrhée': 213125.0,
  'Palu': 1588370.0},
 {'LIBELLE_REGION': 'FITOVINANY',
  'IRA': 93790.0,
  'Diarrhée': 55380.0,
  'Palu': 403595.0},
 {'LIBELLE_REGION': 'VATOVAVY ',
  'IRA': 136990.0,
  'Diarrhée': 87755.0,
  'Palu': 438015.0}]